# Setup

Check allocated GPU, hope for something better than a K80...

In [ ]:
!nvidia-smi

Install the needed python packages

In [2]:
!pip3 install --quiet transformers datasets tokenizers sacrebleu wandb

Clone our repository to get our utilities and overrides

In [ ]:
!git clone https://github.com/lucarinelli/conditional_text_generation.git

Add our repository `src` folder to python path

In [3]:
import sys
import os

module_path = os.path.abspath("/content/conditional_text_generation/src")
if module_path not in sys.path:
    sys.path.append(module_path)

Setup and connect to Weights and Biases to store logs and results

**REMEMBER TO SET WANDB_PROJECT TO THE CORRECT VALUE**

In [4]:
import wandb

%env WANDB_PROJECT=ctrl_dry_runs
%env WANDB_ENTITY=polito_aiml2021_textgen
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
%env WANDB_SILENT=true

wandb.login()

env: WANDB_PROJECT=ctrl_dry_runs
env: WANDB_ENTITY=polito_aiml2021_textgen
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all
env: WANDB_SILENT=true


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

Set training arguments and other experiment parameters

In [ ]:
from experiment_parameters import ExperimentParameters
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./data/results",  # output directory
    save_total_limit=3,
    num_train_epochs=3,  # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=1,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
    logging_dir='./data/logs',  # directory for storing logs
    evaluation_strategy="epoch",
    report_to="wandb",
    load_best_model_at_end=True,
    remove_unused_columns=False
)

experiment_parameters = ExperimentParameters(training_args=training_args)

# Dataset
We download and load the COCO captions dataset.

We join in a single item the caption for an image with the categories and/or supercategories associated to objects present in the image.
Categories and/or supercategories are used as control codes depending on the experiment settings.

The dataset is post processed to train the model with different combinations of control codes for each caption, depending on the experiment parameters. The output of the postprocessing is saved on .json files that are then loaded and further handled by the Dataset class provided by HuggingFace datasets (used for its performance and caching abilities).

In [12]:
from captions_dataset import load_our_dataset

dataset_train, dataset_val = load_our_dataset(experiment_parameters, data_path="/content/data")

Dataset json file does not exist, creating dataset from scratch...
Discarded: 318
There are 4084 captions without a category
Dataset json file does not exist, creating dataset from scratch...
Discarded: 14
There are 192 captions without a category
There are 469064 captions considered in total (train)
There are 19808 captions considered in total (val)
The following 12 categories are present in the dataset:
['kitchen', 'food', 'animal', 'furniture', 'indoor', 'accessory', 'person', 'vehicle', 'outdoor', 'sports', 'appliance', 'electronic']
Processed control codes:
['<CTRL:kitchen>', '<CTRL:food>', '<CTRL:animal>', '<CTRL:furniture>', '<CTRL:indoor>', '<CTRL:accessory>', '<CTRL:person>', '<CTRL:vehicle>', '<CTRL:outdoor>', '<CTRL:sports>', '<CTRL:appliance>', '<CTRL:electronic>']


TypeError: 'str' object cannot be interpreted as an integer